# Intro to Szegedy quantum walks

**Contents**

1. Random walks basics
2. Metropolis-Hastings algorithm

In [33]:
import numpy as np
import networkx as nx
from qiskit.circuit import QuantumCircuit, Gate
from qiskit.circuit.library import StatePreparation
from qiskit.quantum_info import Operator

## Random walks basics

A finite random walk is specified by a transition matrix $P \in \mathbb{R}^{d\times d}$, where
$P_{ij} = \Pr(X_{t+1}=j \mid X_t=i)$.
To ensure that $P$ defines a well-behaved Markov chain, we verify few structural properties.

### Row-stochasticity

Firstly, each row of $P$ must represent a probability distribution, i.e. the matrix is row-stochastic:
$$
P_{ij} \ge 0 \quad \text{and} \quad \sum_j P_{ij} = 1 \quad \forall i.
$$
This condition ensures that if we start from a valid distribution $\pi$, the step $P \pi$ is still a valid distribution. 


In [17]:
def check_row_stochasticity(P, tol=1e-10):
    if (P < 0).any() or not np.allclose(P.sum(1), 1, atol=tol):
        raise ValueError("Invalid MC.")

### Irreducibility 

Secondly, we want the walk is to be irreducible, i.e. if every state can be reached from every other state with nonzero probability in some number of steps. Irreducibility ensures the existence and uniqueness of a stationary distribution $\pi = P \pi$. 

This can be implemented by noticing the condition is equivalent to checking that the directed graph with edges $i \to j$ whenever $P_{ij} > 0$ is strongly connected. 

In [18]:
def check_irreduciblility(P, tol=1e-10):
    if not nx.is_strongly_connected(nx.from_numpy_array(P > tol, create_using=nx.DiGraph)):
        raise ValueError("Chain is not irreducible.")

### Reversibility and detailed balance

An (irreducible) random walk has a unique equilibrium distribution $\pi$ such that, for every pair of states $i,j$,
$$
\pi_i P_{ij} = \pi_j P_{ji}.
$$
The equation above is called *detailed balance*.

We can test reversibility in different ways. One approach is to compute the stationary distribution $\pi$ numerically (via power iteration) and then verify the detailed balance equations directly. However, this approach has downsides: convergence may be slow for nearly reducible chains, and small numerical errors in $\pi$ are amplified when multiplied by transition probabilities.

Note that a finite Markov chain is reversible if and only if, for every closed cycle $i_0 \to i_1 \to \cdots \to i_k \to i_0$, 
the probability of traversing the cycle in the forward direction equals the probability of traversing it in reverse: $\prod_t P_{i_t,i_{t+1}} = \prod_t P_{i_{t+1},i_t}$. Taking logarithms converts products into sums, yielding a more numerically stable test. 

Note also that a symmetric $P$ is also reversible as satisfy the detailed balance by construction, albeit the condition is not needed in general for reversibility. 

In [19]:
def check_reversibility(P, tol=1e-10):
    G = nx.from_numpy_array(P > tol, create_using=nx.Graph)
    for cycle in nx.cycle_basis(G):
        fwd = bwd = 0.0
        for i in range(len(cycle)):
            a, b = cycle[i], cycle[(i + 1) % len(cycle)]
            if P[a, b] <= tol or P[b, a] <= tol:
                raise ValueError("Non-reversible: missing reverse edge.")
            fwd += np.log(P[a, b])
            bwd += np.log(P[b, a])
        if abs(fwd - bwd) > tol:
            raise ValueError("Detailed balance fails on a cycle.")

### Examples

The former $P$ is row-stochastic, the latter is not as row-0 sums to $0.9\neq 1$.

In [20]:
P_row_ok = np.array([
    [0.2, 0.5, 0.3],
    [0.1, 0.7, 0.2],
    [0.4, 0.1, 0.5],
], dtype=float)

P_row_bad = np.array([
    [0.2, 0.5, 0.2],
    [0.1, 0.7, 0.2],
    [0.4, 0.1, 0.5],
], dtype=float)

In [21]:
check_row_stochasticity(P_row_ok)

In [22]:
check_row_stochasticity(P_row_bad)

ValueError: Invalid MC.

The former $P$ is irreducible (a simple 3-cycle with self-loops), the latter is not as {0,1} and {2} form separate classes.

In [24]:
P_irred_ok = np.array([              # strongly connected 
    [0.2, 0.8, 0.0],
    [0.0, 0.2, 0.8],
    [0.8, 0.0, 0.2],
], dtype=float)

P_irred_bad = np.array([             # reducible: 
    [0.5, 0.5, 0.0],
    [0.5, 0.5, 0.0],
    [0.0, 0.0, 1.0],
], dtype=float)

In [25]:
check_irreduciblility(P_irred_ok)

In [27]:
check_irreduciblility(P_irred_bad)

ValueError: Chain is not irreducible.

The former $P$ is reversible (symmetric implies reversibility), the latter is not (violates cycle condition).

In [30]:
P_rev_ok = np.array([                # symmetric => reversible (uniform stationary distribution works)
    [0.5, 0.5, 0.0],
    [0.5, 0.0, 0.5],
    [0.0, 0.5, 0.5],
], dtype=float)

P_rev_bad = np.array([               # has both directions on edges, but violates cycle condition
    [0.0, 0.9, 0.1],
    [0.5, 0.0, 0.5],
    [0.9, 0.1, 0.0],
], dtype=float)

In [31]:
check_reversibility(P_rev_ok)

In [32]:
check_reversibility(P_rev_bad)

ValueError: Detailed balance fails on a cycle.

### Metropolis-Hastings algorithm

A class of MC that obeys detailed balance by construction. The transition probability $x \to y$ is broken into two steps:
* a transition $x \to y$ is proposed, $y \neq x$;
* such a transition is accepted probabilistically with probability $A_{yx}$.
The overall transition probability is
$$P_{yx} = \begin{cases} T_{yx} A_{yx}, & x \neq y \\ 1 - \sum_y P_{yx} A_{yx}, & x = y \end{cases}$$

The detailed balance condition becomes
$$ R_{xy} := \frac{A_{xy}}{A_{yx}} = \frac{\pi_y}{\pi_x} \frac{P_{xy}}{P_{yx}} \quad\Rightarrow\quad A_{yx} = \min(1, R_{xy})$$
Note the above choice of $A_{yx}$ is not the only one. At this point, one still don't know how to calculate $R_{xy}$. 

Let's restrict to Boltzmann distribution. Given an energy function $E(x)$ and inverse temperature $\beta$, we have $\pi_\beta = Z(\beta)^{-1} \, \exp(-\beta E(x))$. In this setting, if we impose the additional condition of symmetry of $T$, i.e. $T_{xy} = T_{yx}$, then the acceptance probability $A_{yx}$ depends uniquely on the difference of energies (which we can now calculate assuming $E$ is a sum of poly many linear terms in the number of vars):
$$A_{yx} = \min(1, e^{\beta |E(x) - E(y)|})$$

## Szegedy quantum walk

The aim here is taking a reversible Markov chain $P$ on $d$ states and embedding into a larger unitary $W$ such that the eigenvalues of $W$ are related to the singular values of $P$. 

1. The workspace is the Hilbert space $\mathcal{H} = \mathbb{C}^d \otimes \mathbb{C}^d$ where states $\ket{x}\ket{y}$ represents a directed edge $x \to y$ if exists
2. For each $x$, the "coin" state $\ket{\psi_x} = \ket{x} \sum_{y} \sqrt P_{yx} \ket{y}$ encodes the probability distribution $P(x, \cdot)$ in the amplitudes.
3. One can pack the information for all $x$ into an isometry $V: \mathbb{C}^d \to \mathcal H$ such that $V \ket{x} = \ket{\psi_x}$. Note $V^\dagger V = I$. The "outgoing" subspace is $A = \mathrm{Im}(V) = \mathrm{span}(\{\ket{\psi_x}\})$ and induces the projector $\Pi_A$.
4. Let $\Lambda$ be the swap operator, then $B = 

## Representing $P$ in Szegedy's format

Szegedy's construction does not act directly on the transition matrix $P$ as a linear operator on probability vectors. We embed it into an operator $W$ acting on the enlarger Hilbert space $\mathcal{H} = \mathbb{C}^d \otimes \mathbb{C}^d$ to enforce the unitarity of the transformation. In this space, basis states $|x\rangle|y\rangle$ can be interpreted as directed edges from $x$ to $y$.

### Definition

The oracle $W$ is defined by its action on states of the form $\ket{x}\ket{0}$ and acts like
$$\ket{x}\ket{0} \mapsto \ket{w_x} \ket{x}, \qquad |w_x\rangle = \sum_{y=0}^{d-1} \sqrt{P_{x y}} \, |y\rangle$$

By convention, the first register holds the result of $P x$ and the second register starts clean then holds the input that once was in the first register. This "swap" step is a convention and you can of course re-define the framework to work with transformation in the form $\ket{x}\ket{0} \mapsto \ket{x} \ket{w_x}$.

### Isometry to unitarity

The operator $W$ appearing in Szegedy's construction is not defined as a unitary on the full Hilbert space. Instead, it is first specified as an *isometry*. Concretely, $W$ is defined by its action on the subspace spanned by basis states $|x\rangle|0\rangle$,
$$
W:\ |x\rangle|0\rangle \longmapsto |w_x\rangle|x\rangle,
\qquad
|w_x\rangle = \sum_y \sqrt{P_{xy}}\,|y\rangle.
$$
This mapping preserves inner products on its domain and therefore satisfies $W^\dagger W = I$ on that subspace. Since the output states $|w_x\rangle|x\rangle$ live in a larger Hilbert space, $W$ enlarges the space and is not invertible on its image.

To use $W$ in a quantum circuit, it must be implemented as a unitary operator acting on the full register space. This is achieved by extending the isometry to a unitary: the action of $W$ is fixed on the subspace $\{|x\rangle|0\rangle\}$, while its action on the orthogonal complement is defined so that the overall operator is unitary. 

### Implementation in Qiskit

The circuit implements $W$ by first swapping the two $n$-qubit registers so $|x\rangle|0\rangle \mapsto |0\rangle|x\rangle$, then looping over each row $x$ of $P$ and building an amplitude vector with entries $\sqrt{P[x,y]}$ (zero-padded to length $2^n$ and normalized). It appends a `StatePreparation` gate for this vector, multi-controlled on the second register being $|x\rangle$, so that only in that case the first register is prepared as $|w_x\rangle=\sum_y \sqrt{P[x,y]}|y\rangle$.

If you want to avoid StatePreparation, the most direct construction is a cascade of multi-controlled $R_y$ rotations where each rotation sets a conditional probability of one output bit given the previously prepared bits.

In [37]:
class SzegedyW(Gate):
    """
    Oracle W (Eq. 2 arXiv:1910.01659):
        W|x>|0> = |w_x>|x>,   |w_x> = sum_y sqrt(P[x,y]) |y|
    Acts on 2n qubits, n = ceil(log2(d)).
    """

    def __init__(self, P, tol=1e-10, label=None):
        self._validate(P, tol)
        self.P, self.tol = P, tol
        self.d = P.shape[0]
        self.n = int(np.ceil(np.log2(self.d))) if self.d > 1 else 1
        self.dim = 1 << self.n
        super().__init__("SzegedyW", 2 * self.n, [], label=label)
        self.definition = self._build_definition()

    def _validate(self, P, tol):
        if P.ndim != 2 or P.shape[0] != P.shape[1]:
            raise ValueError("P must be square.")
        check_row_stochasticity(P, tol)
        check_irreduciblility(P, tol)
        check_reversibility(P, tol)
    
    def _build_definition(self):
        qc = QuantumCircuit(2 * self.n, name="SzegedyW")
        A, B = range(self.n), range(self.n, 2 * self.n)

        # |x>|0> → |0>|x>
        for k in range(self.n):
            qc.swap(A[k], B[k])

        # StatePreparation for each |w_x>
        for x in range(self.d):
            if not (self.P[x] > self.tol).any():
                continue
            amps = np.zeros(self.dim)
            amps[:self.d] = np.sqrt(self.P[x])
            amps /= np.linalg.norm(amps)
            qc.append(
                StatePreparation(amps).control(self.n, ctrl_state=x),
                list(B) + list(A),
            )

        return qc

In [54]:
# 1) Define a small P that passes row-stochasticity + irreducibility + reversibility (symmetric => reversible)
P = np.array([
    [0.5, 0.5, 0.0],
    [0.5, 0.0, 0.5],
    [0.0, 0.5, 0.5],
], dtype=float)

# 2) Instantiate a circuit and apply SzegedyW
W = SzegedyW(P)
qc = QuantumCircuit(2 * W.n)
qc.append(W, range(2 * W.n))

# 3) Get the unitary matrix
U = Operator(qc).data
print(f"U shape: {U.shape} | n: {W.n} qubits can fit {2**W.n} >= {P.shape[0]} states")  # should be (2^(2n), 2^(2n))

# 4) In Qiskit, when you convert a circuit to a matrix, rows/columns are indexed by the computational basis integer of the whole register. 
# With registers ordered as: first register A in qubit 0...n-1, second register in B in qubit n...2n-1 the basis state |y>_A|x>_B is y + 2^n x
# np.sqrt -> entry-wise square root
x = np.arange(W.d)[:, None]
y = np.arange(W.d)[None, :]
sqrtP_from_W = U[y + x * (2**W.n), x]
print(np.allclose(sqrtP_from_W, np.sqrt(P), atol=1e-8))

U shape: (16, 16) | n: 2 qubits can fit 4 >= 3 states
True


## Szegedy's quantum walk

Let $\mathcal{H} = \mathbb{C}^d \otimes \mathbb{C}^d$ be the Hilbert space on which the Szegedy quantum walk is defined. Let
$$
\mathcal{E}_0 = \operatorname{span}\{\, |x\rangle|0\rangle \mid x = 0,\dots,d-1 \,\}
$$
be the subspace in which the second register is clean. The projector onto this subspace is
$$
\Pi_0 = \mathbb{I} \otimes |0\rangle\langle 0|
= \sum_{x=0}^{d-1} |x\rangle\langle x| \otimes |0\rangle\langle 0|,
$$
and the corresponding reflection is
$$
R_0 = 2 \Pi_0 - \mathbb{I}.
$$

### Projection on the outgoing transitions

Using the isometry, define the projector
$$
\Pi_A = W \Pi_0 W^\dagger.
$$
The image of $\Pi_A$ is the subspace spanned by the states $|w_x\rangle|x\rangle$, which encode the outgoing transitions of the Markov chain. The associated reflection is
$$
R_A = 2\Pi_A - \mathbb{I}.
$$
This reflection acts as the identity on states in the image of $\Pi_A$ and flips the sign of states orthogonal to that image.

### Projection on the ingoing transitions

Let $\Lambda$ denote the swap operator between the two registers,
$$
\Lambda\,|x\rangle|y\rangle = |y\rangle|x\rangle.
$$
Conjugating $\Pi_A$ by $\Lambda$ defines a second projector,
$$
\Pi_B = \Lambda \Pi_A \Lambda,
$$
whose image is the subspace spanned by $|x\rangle|w_x\rangle$. This subspace coherently encodes the incoming transitions of the Markov chain. The corresponding reflection is
$$
R_B = 2\Pi_B - \mathbb{I} = \Lambda R_A \Lambda.
$$

### Walk operator

The Szegedy quantum walk operator is then defined as the product of these two reflections:
$$
U = R_B R_A
$$

As a product of reflections, $U$ is unitary and acts as a rotation on invariant two-dimensional subspaces determined by the overlap between the outgoing- and incoming-transition subspaces. When the underlying Markov chain is reversible, the spectrum of $U$ is directly related to the spectrum of the classical transition matrix $P$.

### Implementation in Qiskit

In [56]:
class Reflection0(Gate):
    r"""
    R0 = 2 Π0 - I,  Π0 = I ⊗ |0...0><0...0|
    Acts on 2n qubits: A[0..n-1], B[n..2n-1].
    """
    def __init__(self, n, label=None):
        self.n = n
        super().__init__("R0", 2 * n, [], label=label)
        self.definition = self._build_definition()

    def _build_definition(self):
        qc = QuantumCircuit(2 * self.n, name="R0")
        B = list(range(self.n, 2 * self.n))

        if self.n == 1:
            qc.z(B[0]) # Special case: I_A ⊗ Z_B  == diag(+1, -1) on B
            return qc

        # Implement phase flip on |0...0>_B
        # I feel we should add a global phase pi to get +1 on |0...0>, -1 otherwise? 
        # Not really used here but if we recycle this code in a controlled-R0 might come in handy... open to feedback
        qc.x(B)
        qc.mcp(np.pi, B[:-1], B[-1])   # adds -1 phase on |1...1>_B
        qc.x(B)
        qc.global_phase += np.pi # multiply whole operator by -1  => reflection 2|0><0| - I on B

        return qc

class LambdaSwap(Gate):
    r"""
    Λ: swap the two n-qubit registers.
    Acts on 2n qubits: A[0..n-1], B[n..2n-1].
    """
    
    def __init__(self, n, label=None):
        self.n = n
        super().__init__("Λ", 2 * n, [], label=label)
        self.definition = self._build_definition()

    def _build_definition(self):
        qc = QuantumCircuit(2 * n, name="Λ")
        A = list(range(n))
        B = list(range(n, 2 * n))
        for k in range(n):
            qc.swap(A[k], B[k])
        return qc

In [69]:
n = W.n
R0 = Reflection0(n)
R0_actual = Operator(R0).data
R0_expected = np.diag([1] * (2**n) + [-1] * (2**(2*n) - 2**n)) # R0 = +1 on |a>|b=0...0>, and -1 otherwise.
np.allclose(R0_actual, R0_expected, atol=1e-8)

True

In [75]:
N = 2**(2*n)
Lambda = LambdaSwap(n)
Lambda_actual = Operator(Lambda).data
idx = np.arange(N)
out = (idx // 2**n) + 2**n * (idx % 2**n) # swap A (low bits) and B (high bits)
Lambda_expected = np.eye(N, dtype=complex)[out]
np.allclose(Lambda_actual, Lambda_expected, atol=1e-8)

True

The quantum walk operator is obtained by composing the elements defined above.

In [80]:
qargs = list(range(2 * n))

qc_RA = QuantumCircuit(2 * n, name="RA")
qc_RA.append(W, qargs)
qc_RA.append(R0, qargs)
qc_RA.append(W.inverse(), qargs)

qc_RB = QuantumCircuit(2 * n, name="RB")
qc_RB.append(Lambda, qargs)
qc_RB.append(qc_RA.copy(), qargs)
qc_RB.append(Lambda, qargs)

qc_U = QuantumCircuit(2 * n, name="U")
qc_U.append(qc_RA.copy(), qargs)
qc_U.append(qc_RB.copy(), qargs)
qc_U.decompose().draw()

┌───────────┐┌─────┐┌──────────────┐┌────┐┌─────┐┌────┐
q_0: ┤0          ├┤0    ├┤0             ├┤0   ├┤0    ├┤0   ├
     │           ││     ││              ││    ││     ││    │
q_1: ┤1          ├┤1    ├┤1             ├┤1   ├┤1    ├┤1   ├
     │  Szegedyw ││  R0 ││  SzegedyW_dg ││  Λ ││  RA ││  Λ │
q_2: ┤2          ├┤2    ├┤2             ├┤2   ├┤2    ├┤2   ├
     │           ││     ││              ││    ││     ││    │
q_3: ┤3          ├┤3    ├┤3             ├┤3   ├┤3    ├┤3   ├
     └───────────┘└─────┘└──────────────┘└────┘└─────┘└────┘

## Spectral analysis of the walk operator

The spectral analysis of the walk operator $U$ is facilitated by the following procedures:

* there exists a unitary operator $U_W$ such that $U$ is unitarily similar to $U_W^2$, thus the latter twos share the same spectrum;
* the operator $U_W$ written in the basis of $\mathcal{H} = \mathcal{E}_0 \oplus \mathcal{E}_0^\bot$ is such that the top-left $d \times d$ block, denoted here with $X$, determines the spectrum of $U_W$;
* the matrices $X$ and $P$ are similar and thus have the same eigenvalues.

It is much easier to study the spectrum of (the $d \times d$ matrix) $P$ than the spectrum of ($d^2 \times d^2$) $U$. 

### The half-step unitary $U_W$

Let's define the "half-step" unitary 
$$U_W = R_0 W^\dagger \Lambda W$$

**Statement**: It holds that $U_W^2$ is unitary similar to $U$, thus share the same spectrum. 

**Proof**: 
$$\begin{align*}
    (\Lambda W) U_W^2 (\Lambda W)^\dagger 
    & = \Lambda W U_W^2 W^\dagger \Lambda \\
    & = \Lambda W [R_0(W^\dagger \Lambda W) \, R_0(W^\dagger \Lambda W)] W^\dagger \Lambda \\
    & = \Lambda W R_0 W^\dagger \Lambda W R_0 (W^\dagger \Lambda W W^\dagger \Lambda) \\
    & = \Lambda W R_0 W^\dagger \Lambda W R_0 W^\dagger \\
    & = \Lambda W (R_0 W^\dagger \Lambda W) R_0 W^\dagger \\
    & = \Lambda W \; U_W \; R_0 W^\dagger \\
    & = U 
\end{align*}$$

In [84]:
mW   = Operator(W).data
mLam = Lambda_actual
mR0  = R0_actual

# Walk operator U = R_B R_A with R_A = W R0 W† and R_B = Λ R_A Λ
mRA = mW @ mR0 @ mW.conj().T
mRB = mLam @ mRA @ mLam
mU  = mRB @ mRA

# Half-step unitary U_W = R0 W† Λ W
mUW = mR0 @ mW.conj().T @ mLam @ mW

# Similarity transform: (ΛW) U_W^2 (ΛW)† should equal U
mLW  = mLam @ mW
mU_equiv = mLW @ (mUW @ mUW) @ mLW.conj().T

np.allclose(mU, mU_equiv, atol=1e-10)

True

### The top-left block $X$ and the similarity to $P$

Let $\Pi_0$ projects onto $\mathcal E_0=\mathrm{span}\{|x\rangle|0\rangle\}$ and $\Lambda$ swaps the two registers. Define:
$$
X := \Pi_0\,W^\dagger \Lambda W\,\Pi_0,
$$

**Statement**: $X$ and $P$ are similar, thus share the same spectrum. 

**Proof**: By construction, $X$ acts only on $\mathcal E_0$ (dimension $d$), and its matrix elements in the basis $\{|x\rangle|0\rangle\}$ are
$$
[X]_{yx} \;=\; \langle y,0|\,W^\dagger \Lambda W\,|x,0\rangle.
$$
Using $W|x,0\rangle = |w_x\rangle|x\rangle$ with $|w_x\rangle=\sum_{y}\sqrt{P_{xy}}\,|y\rangle$ and $\Lambda|a\rangle|b\rangle=|b\rangle|a\rangle$, one obtains
$$
[X]_{yx} \;=\; \langle w_y|x\rangle\,\langle y|w_x\rangle
\;=\;\sqrt{P_{yx}}\sqrt{P_{xy}}
$$

Assuming reversibility, there is an unique distribution $\pi$ such that $\pi_x P_{xy} = \pi_y P_{yx}$. Therefore, $P_{yx} = (\pi_x/\pi_y)P_{xy}$ and hence
$$
[X]_{xy} = \sqrt{P_{xy}P_{yx}} = P_{xy}\sqrt{\frac{\pi_x}{\pi_y}}.
$$

Let $D:=\mathrm{diag}(\sqrt{\pi_0},\dots,\sqrt{\pi_{d-1}})$. The previous identity is exactly
$$
X = D\,P\,D^{-1}.
$$


In [92]:
d = P.shape[0]

# T = W† Λ W
T = mW.conj().T @ mLam @ mW

# Π0 = I ⊗ |0...0><0...0|  (project onto B=0 block of size 2^n)
Pi_0 = np.zeros((N, N), dtype=complex)
Pi_0[:2**n, :2**n] = np.eye(2**n, dtype=complex)

# X = Π0 T Π0, then restrict to the physical d-dimensional subspace (x = 0..d-1)
X_full = Pi_0 @ T @ Pi_0
X = X_full[:d, :d]

# 1) Check the entrywise formula:  X_{yx} = sqrt(P_{yx} P_{xy})
X_expected = np.sqrt(P * P.T)
print("X matches sqrt(P_xy P_yx):", np.allclose(X, X_expected, atol=1e-10))

# 2) Compute stationary distribution π from P^T π = π and verify X = D P D^{-1}
evals, evecs = np.linalg.eig(P.T)
k = np.argmin(np.abs(evals - 1.0))
pi = np.real(evecs[:, k])
pi = np.abs(pi) / np.sum(np.abs(pi))

D = np.diag(np.sqrt(pi))
X_sim = D @ P @ np.linalg.pinv(D)
print("X matches D P D^{-1}:", np.allclose(X, X_sim, atol=1e-10))

X matches sqrt(P_xy P_yx): True
X matches D P D^{-1}: True


### The top-left block $X$ and connection to $U_W$

The operator $U_W = R_0 W^\dagger \Lambda W$ can be written as the composition of two rotations (operators squaring to the identity), one being $R_0$ and the other being $T := W^\dagger \Lambda W$ (since $\Lambda$ is Hermitian unitary and $W$ is unitary).

----

**Jordan’s lemma**: Since $R_0^2=T^2=I$, there exists an orthogonal decomposition
$$
\mathcal H \;=\;\Big(\bigoplus_{k\in\mathcal K} \mathcal H_k\Big)\;\oplus\;\mathcal H_{\pm},
\qquad \dim(\mathcal H_k)=2,
$$
such that each $\mathcal H_k$ is invariant under both $R_0$ and $T$ (hence also under $U_W=R_0T$), while $\mathcal H_{\pm}$ further decomposes into 1D invariant subspaces on which $U_W$ acts as $\pm 1$. Equivalently, in a suitable orthonormal basis,
$$
U_W \;\simeq\; \Big(\bigoplus_{k\in\mathcal K} U_W^{(k)}\Big)\;\oplus\;(\pm 1)\;\oplus\;(\pm 1)\;\oplus\cdots,
$$
with $2\times 2$ blocks $U_W^{(k)}$ and the remaining blocks of size $1\times 1$ equal to $+1$ or $-1$.

----



**Statement**: the eigenvalues of $U_W$ are either $\pm 1$ or $e^{\pm i\arccos(\lambda_k)}$ for $\lambda_k \in \mathrm{spec}(X)$.

**Proof**: Write the Hilbert space as $\mathcal{H}=\mathcal{E}_0\oplus\mathcal{E}_0^\bot$. In this basis
$$
R=\begin{pmatrix}I&0\\0&-I\end{pmatrix},\qquad
T=\begin{pmatrix}X&Y\\Y^\dagger&Z\end{pmatrix},
\qquad\Rightarrow\qquad
U_W=RT=\begin{pmatrix}X&Y\\-Y^\dagger&-Z\end{pmatrix}.
$$

Since $T$ is a reflection, $T^2=I$, hence
$$
\begin{pmatrix}
X^2+YY^\dagger & XY+YZ\\
Y^\dagger X+ZY^\dagger & Y^\dagger Y+Z^2
\end{pmatrix}
=
\begin{pmatrix}I&0\\0&I\end{pmatrix}
\quad\Rightarrow\quad
YY^\dagger=I-X^2,\;\; Y^\dagger X+ZY^\dagger=0.
$$

We seek, for each eigenvalue $\lambda_k$ of $X$, two vectors $|v_k\rangle\in\mathcal{E}_0$ and $|w_k\rangle\in\mathcal{E}_0^\bot$ such that $\mathrm{span}\{|v_k\rangle,|w_k\rangle\}$ is invariant under $U_W$, so that $U_W$ reduces to a $2\times2$ block on that plane. 

1. Let $X v_k=\lambda_k v_k$ with $\|v_k\|=1$, and embed $v_k$ as $|v_k\rangle:=\begin{bsmallmatrix}v_k\\0\end{bsmallmatrix}$. Then
$$
U_W|v_k\rangle=\begin{pmatrix}\lambda_k v_k\\-Y^\dagger v_k\end{pmatrix}.
$$

2. Define $|w_k\rangle:=\frac{1}{\beta_k}\begin{bsmallmatrix}0\\Y^\dagger v_k\end{bsmallmatrix}$ where $\beta_k:=\|Y^\dagger v_k\|$.
    * Using $YY^\dagger=I-X^2$, for $|\lambda_k|<1$ we have
$$
\beta_k^2=\langle v_k|YY^\dagger|v_k\rangle=\langle v_k|(I-X^2)|v_k\rangle=1-\lambda_k^2,
\qquad \Rightarrow \qquad \beta_k=\sqrt{1-\lambda_k^2}.
$$
    * The action of $U_W$ on $|v_k\rangle$ lies in $\mathrm{span}\{|v_k\rangle,|w_k\rangle\}$:
$$
U_W|v_k\rangle=\lambda_k|v_k\rangle-\beta_k|w_k\rangle.
$$
    * The action of $U_W$ on $|w_k\rangle$ lies in $\mathrm{span}\{|v_k\rangle,|w_k\rangle\}$ as well. Start from the definition of $|w_k\rangle$ and apply $U_W$:
$$
\begin{align*}
U_W|w_k\rangle
&= \frac{1}{\beta_k}\,U_W\begin{pmatrix}0\\Y^\dagger v_k\end{pmatrix}
= \frac{1}{\beta_k}\begin{pmatrix}YY^\dagger v_k\\-ZY^\dagger v_k\end{pmatrix}.
\end{align*}
$$
        Now:
        - Top: $YY^\dagger v_k=(I-X^2)v_k=(1-\lambda_k^2)v_k=\beta_k^2 v_k$.
        - Bottom: from $Y^\dagger X+ZY^\dagger=0$ we get $ZY^\dagger=-Y^\dagger X$, hence
          $-ZY^\dagger v_k=Y^\dagger X v_k=\lambda_k Y^\dagger v_k$.
        - Substituting and using $Y^\dagger v_k=\beta_k\,w_k$ gives
        $$
        \begin{align*}
        U_W|w_k\rangle
        &=\frac{1}{\beta_k}\begin{pmatrix}\beta_k^2 v_k\\ \lambda_k Y^\dagger v_k\end{pmatrix}
        =\begin{pmatrix}\beta_k v_k\\ \lambda_k\,(\frac{1}{\beta_k}Y^\dagger v_k)\end{pmatrix}
        =\beta_k|v_k\rangle+\lambda_k|w_k\rangle.
        \end{align*}
        $$

We conclude that, on $\mathrm{span}\{|v_k\rangle,|w_k\rangle\}$, in the ordered basis $\{|v_k\rangle,|w_k\rangle\}$,
$$
U_W^{(k)}=\begin{pmatrix}\lambda_k & \beta_k \\ -\beta_k & \lambda_k\end{pmatrix}
=\begin{pmatrix}\lambda_k & \sqrt{1-\lambda_k^2} \\ -\sqrt{1-\lambda_k^2} & \lambda_k\end{pmatrix},
$$
with eigenvalues $\lambda_k \pm i\sqrt{1-\lambda_k^2} = e^{\pm i\arccos(\lambda_k)}$.

(If $|\lambda_k|=1$ then $\beta_k=0$, $Y^\dagger v_k=0$, and $U_W|v_k\rangle=\lambda_k|v_k\rangle$ gives the $\pm1$ eigenvalues on 1D invariant subspaces.)


In [106]:
tol = 1e-8
eigUW = np.linalg.eigvals(mUW)
eigX  = np.linalg.eigvals(X)

# for each mu_k in eigUW check either +- 1 or exp(pm i arccos(lambda_k))
lam = np.clip(np.real_if_close(eigX, tol=1e6).astype(float), -1.0, 1.0)
theta = np.arccos(lam)

for mu in eigUW:
    if abs(mu) - 1.0 < tol:
        continue # ok because pm 1 or pm i. 
        # If you use X_full instead of X, the zero eigenvals in X_full are the only ones that becomes pm i 
        # and are catched in the next if clause. Therefore this clause can become abs(mu - 1) < tol or abs(mu + 1) < tol
    if any(abs(mu - np.exp(1j*t)) < tol or abs(mu - np.exp(-1j*t)) < tol for t in theta):
        continue # exp(pm i arccos(lambda_k))
    raise ValueError("Invalid eigenvalue", mu)

## Connection to QSVT

TODO many steps of Szegedy's walk makes a cheby's poly of the first kind, you can compose them via LCU to make a poor's man QSVT on $P$. 